In [2]:
import requests
from bs4 import BeautifulSoup


In [3]:
from google.colab import auth
auth.authenticate_user()

In [4]:
from google.auth import default
import gspread
creds, _ = default()
gc = gspread.authorize(creds)

In [8]:
worksheet = gc.open('webData').sheet1
list_of_dicts = worksheet.get_all_records()
print(list_of_dicts)

[{'Links': 'https://islamqa.info/en/answers/1/when-can-you-pray-after-a-miscarriage', 'Publication Date': '', 'Views': '', 'Question': ''}, {'Links': 'https://islamqa.info/en/answers/2/when-can-you-pray-after-a-miscarriage', 'Publication Date': '', 'Views': '', 'Question': ''}, {'Links': 'https://islamqa.info/en/answers/3/when-can-you-pray-after-a-miscarriage', 'Publication Date': '', 'Views': '', 'Question': ''}, {'Links': 'https://islamqa.info/en/answers/4/when-can-you-pray-after-a-miscarriage', 'Publication Date': '', 'Views': '', 'Question': ''}]


In [9]:
import pandas as pd
df = pd.DataFrame(list_of_dicts)
df.head()


,Links,Publication Date,Views,Question
0,https://islamqa.info/en/answers/1/when-can-you...,,,
1,https://islamqa.info/en/answers/2/when-can-you...,,,
2,https://islamqa.info/en/answers/3/when-can-you...,,,
3,https://islamqa.info/en/answers/4/when-can-you...,,,


In [10]:
def Scrap_Data(url):
  page=requests.get(url)
  soup=BeautifulSoup(page.content,'html.parser')
  elements = soup.find_all("div", class_="level-item has-separator is-hidden-mobile")
  publication=elements[0].text.replace(" ","").replace("\n","")[12:]
  views=elements[1].text.replace(" ","").replace("\n","")[6:]
  ques_ans=soup.find_all("section")
  question=ques_ans[0].text.replace('\n','').replace('  ','')[8:]

  return [publication,views,question]

In [12]:
for idx in range(len(df)):
  link=df.loc[idx]["Links"]
  output=Scrap_Data(link)
  p,v,q=output[0],output[1],output[2]

  df.loc[idx]["Publication Date"]=p
  df.loc[idx]["Views"]=v
  df.loc[idx]["Question"]=q


In [13]:
df

,Links,Publication Date,Views,Question
0,https://islamqa.info/en/answers/1/when-can-you...,29-05-1997,54047,"If during wudhu (ablution), one finds dirt stu..."
1,https://islamqa.info/en/answers/2/when-can-you...,29-05-1997,28724,What should a person do if one of the areas no...
2,https://islamqa.info/en/answers/3/when-can-you...,29-05-1997,39929,If a man sees the signs of sexual discharge im...
3,https://islamqa.info/en/answers/4/when-can-you...,29-05-1997,44003,A lawyer in Argentina is asking about the ruli...


In [16]:
df.to_csv('/content/drive/MyDrive/webData.csv', index=False)